# Abstractive HuggingFace

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/augmentation-abstractive](https://github.com/huseinzol05/Malaya/tree/master/example/augmentation-abstractive).
    
</div>

<div class="alert alert-warning">

Results generated using stochastic methods.
    
</div>

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
%%time

import malaya

CPU times: user 3.73 s, sys: 3.73 s, total: 7.47 s
Wall time: 3.04 s


### Why augmentation

Let say you have a very limited labelled corpus, and you want to add more, but labelling is very costly.

So, text augmentation! We provided few augmentation interfaces in Malaya.

### List available HuggingFace models

In [3]:
malaya.augmentation.abstractive.available_huggingface()

,Size (MB),BLEU,SacreBLEU Verbose,Suggested length
mesolitica/finetune-noisy-translation-t5-tiny-bahasa-cased-v3,139,60.000967,77.9/63.9/54.6/47.7 (BP = 1.000 ratio = 1.036 ...,256
mesolitica/finetune-noisy-translation-t5-small-bahasa-cased-v3,242,64.062582,80.1/67.7/59.1/52.5 (BP = 1.000 ratio = 1.042 ...,256
mesolitica/finetune-noisy-translation-t5-base-bahasa-cased-v2,892,64.583819,80.2/68.1/59.8/53.2 (BP = 1.000 ratio = 1.048 ...,256


### Load HuggingFace model

```python
def huggingface(
    model: str = 'mesolitica/finetune-noisy-translation-t5-small-bahasa-cased-v3',
    lang: str = 'ms',
    force_check: bool = True,
    **kwargs,
):
    """
    Load HuggingFace model to abstractive text augmentation.

    Parameters
    ----------
    model: str, optional (default='mesolitica/finetune-noisy-translation-t5-small-bahasa-cased-v4')
        Check available models at `malaya.augmentation.abstractive.available_huggingface()`.
    lang: str, optional (default='ms')
        Input language, only accept `ms` or `en`.
    force_check: bool, optional (default=True)
        Force check model one of malaya model.
        Set to False if you have your own huggingface model.

    Returns
    -------
    result: malaya.torch_model.huggingface.Generator
    """
```

So you can choose either `ms` or `en` input language, for mixed language, it is save to choose both.

In [4]:
model = malaya.augmentation.abstractive.huggingface(model = 'mesolitica/finetune-noisy-translation-t5-small-bahasa-cased-v3')

### Generate

```python
def generate(self, strings: List[str], return_generate=False, **kwargs):
    """
    Generate texts from the input.

    Parameters
    ----------
    strings : List[str]
    **kwargs: vector arguments pass to huggingface `generate` method.
        Read more at https://huggingface.co/docs/transformers/main_classes/text_generation

    Returns
    -------
    result: List[str]
    """
```

In [5]:
string = 'saya suka makan ayam dan ikan'
string2 = 'saya sebenarnya tak suka sangat dekat lelaki tu, ketiak masam sebab tak mandi'
string3 = 'Perdana Menteri berkata, beliau perlu memperoleh maklumat terperinci berhubung isu berkenaan sebelum kerajaan dapat mengambil sebarang tindakan lanjut. Bagaimanapun, beliau yakin masalah itu dapat diselesaikan dan pentadbiran kerajaan boleh berfungsi dengan baik.'

In [6]:
model.generate([string, string2, string3], max_length = 256)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


['suka makan ayam dan ikan',
 'Aku sebenarnya tak suka sangat dekat laki tu, ketiak masam sebab tak mandi',
 'Perdana Menteri berkata, beliau perlu mendapatkan maklumat terperinci berhubung isu tersebut sebelum kerajaan dapat mengambil sebarang tindakan lanjut. Namun beliau yakin masalah tersebut dapat diselesaikan dan pentadbiran kerajaan dapat berfungsi dengan baik.']

### Good thing about HuggingFace

In `generate` method, you can do greedy, beam, sampling, nucleus decoder and so much more, read it at https://huggingface.co/blog/how-to-generate

In [7]:
outputs = model.generate(['rakyat memang tak suka awak pun'], 
                         max_length = 100, do_sample=True, top_k=100, top_p=0.95, temperature=0.7,
                         num_return_sequences=5)
outputs

['rakyat betul2 tak suka kau pun',
 'rakyat mmg tak suka kau pun',
 'rakyat mmg xsuka kau pon',
 'rakyat mmg x suka kau pon',
 'Rakyat mmg tak suka kamu pon']

In [8]:
outputs = model.generate([string, string2, string3], 
                         max_length = 100, do_sample=True, top_k=100, top_p=0.95, temperature=0.7,
                         num_return_sequences=3)
outputs

['suka makan ayam ngan ikan',
 'suka makan ayam dan ikan',
 'suka makan ayam ikan',
 'aku sebenarnya tak suka sangat dekat lelaki tu, ketiak masam sebab tak mandi',
 'Saya sebenarnya tak suka sangat dekat si lelaki tu, ketiak masam sebab tak mandi',
 'aku sebenarnya tak suka sangat dekat sorang laki tu ketiak aku masam sebab tak mandi',
 'Perdana Menteri berkata, beliau perlu mendapatkan maklumat terperinci mengenai isu tersebut sebelum kerajaan dapat mengambil sebarang tindakan lanjut. Bagaimanapun, beliau yakin masalah tersebut dapat diselesaikan dan pentadbiran kerajaan dapat berfungsi dengan baik.',
 'Perdana Menteri berkata, beliau perlu memperoleh maklumat terperinci berhubung isu tersebut sebelum kerajaan dapat mengambil sebarang tindakan lanjut. Bagaimanapun beliau yakin masalah itu dapat diselesaikan dan pentadbiran kerajaan dapat berfungsi dengan baik.',
 'Perdana Menteri berkata, beliau perlu mendapatkan maklumat terperinci berhubung isu tersebut sebelum kerajaan boleh menga

You can choose different generating method and still maintain the same polarity.

In [9]:
strings = [
    'bodoh betul kerajaan ni, mana kebebasan bersuara',
    'Perbincangan khas itu juga bertujuan bagi Seri Paduka mendapat pandangan Raja-Raja Melayu untuk membolehkan baginda membuat keputusan yang terbaik demi kepentingan dan kesejahteraan negara serta rakyat',
    'semalam saya buka tv tengok berita, katanya ada tanah runtuh',
    'i hate him so much, perangai teruk!'
]

In [10]:
outputs = model.generate(strings, 
                         max_length = 100, do_sample=True, top_k=100, top_p=0.95, temperature=0.8,
                         num_return_sequences=3)
outputs

['Bodoh betul kerajaan ni, mana kebebasan bersuara',
 'Bodoh betul kerajaan ni, mana freedom of speech',
 'bodoh betul kerajaan ni,mana kebebasan bersuara',
 'Perbincangan khas itu juga bertujuan Seri Paduka mendapat pandangan Raja2 Melayu untuk membolehkan baginda membuat keputusan yang terbaik demi kepentingan dan kesejahteraan negara serta rakyat',
 'Perbincangan khas itu juga bertujuan agar Seri Paduka mendapat pandangan Raja-raja Melayu bagi membolehkan baginda membuat keputusan yang terbaik demi kepentingan dan kesejahteraan negara serta rakyat',
 'Perbincangan khas tu juga bertujuan bagi Seri Paduka mendapat pandangan Raja-raja Melayu utk membolehkan Baginda membuat keputusan terbaik demi kepentingan dan kesejahteraan negara dan rakyat',
 'Semalam bukak tv tgk news katanya ada landslide',
 'semalam bukak tv tgk berita katanya ada tanah runtuh',
 'semalam bukak tv tengok berita ni katanya ada tanah runtuh',
 'i hate him so much, perangai jahat!',
 'aku benci sangat dia tu, teruk 

In [11]:
outputs = model.generate(strings, 
                         max_length = 100, do_sample=True, penalty_alpha=0.9, top_k=4, temperature=0.9,
                         num_return_sequences=3)
outputs

['Bodo betul kerajaan ni mna kebebasan bersuara',
 'bodo betul kerajaan ni mne kebebasan bersuara',
 'Bodoh betul kerajaan ni,mana kebebasan bersuara',
 'Perbincangan khas itu turut bermatlamat untuk Seri Paduka mendapat pandangan Raja2 Melayu untuk membolehkan Baginda membuat keputusan yang terbaik demi kepentingan dan kesejahteraan Negara dan rakyat',
 'Perbincangan khas tu juga bertujuan untuk Seri Paduka mendapat pandangan Raja-Raja Melayu bagi membolehkan Baginda membuat keputusan terbaik demi kepentingan dan kesejahteraan negara serta rakyat',
 'Perbincangan khas itu juga bertujuan supaya Seri Paduka mendapat pandangan Raja2 Melayu bagi membolehkan baginda membuat keputusan yang terbaik demi kepentingan dan kesejahteraan negara serta rakyat',
 'semalam buka tv tengok news kata kat sana ada landslide',
 'semalam bukak tv tgk berita katanya ade tanah runtuh',
 'Semalam buka tv tengok berita dia kata dah ada tanah runtuh',
 'Aku benci dia sangat, perangai jahat wey!',
 'i hate him s